In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
####Path to code user needs to update these three parameters#####
path_to_code = "/path/to/code/PPIscreenMML/PPIScreenML_model.pkl"
#####path_to_model_code_folder is the path to where the AF models are stored. Does not have to be the same location as the code
path_to_AF_models = "/path/to/AF_models"
#### name of feature_file####
name_of_feature_csv_file = "name_of_output_file".csv"


In [3]:
df = pd.read_csv(f"{path_to_AF_models}/{name_of_feature_csv_file}")

columns_red_feats =  [4, 14, 15, 16, 17, 21, 28]
df_columns_red_feats = [-1, 4, 14, 15, 16, 17, 21, 28]

columns_red_feats_std = [val + 1 for val in columns_red_feats]
df_columns_red_feats_std = [val + 1 for val in df_columns_red_feats]

nonans_red_feats = df.iloc[:, columns_red_feats_std].dropna()
df_nonans_red_feats = df.iloc[:, df_columns_red_feats_std].dropna()


X = nonans_red_feats.values
X_df = df_nonans_red_feats

In [4]:
print(X_df)

                                             File_name  iPAE_1_4  avg_IF_atr  \
0    KLHL12_FL_EPHA3_trimmed_relaxed_rank_002_alpha...   19.0106   -0.136079   
1    DTL_FL_EPHA3_trimmed_relaxed_rank_004_alphafol...   18.6799   -0.076653   
2    DCAF15_cut3_EPHA3_trimmed_relaxed_rank_002_alp...   19.8900   -0.138310   
3    FBXO4_FL_EPHA3_trimmed_relaxed_rank_005_alphaf...   28.0462   -0.055400   
4    KLHL11_FL_EPHA3_trimmed_relaxed_rank_005_alpha...   24.4166   -0.287746   
..                                                 ...       ...         ...   
195  TRPC4AP_FL_EPHA3_trimmed_relaxed_rank_005_alph...   28.9627   -0.001502   
196  KLHL12_FL_EPHA3_trimmed_relaxed_rank_001_alpha...   17.2491   -0.150218   
197  KLHL6_trimmed_EPHA3_trimmed_relaxed_rank_003_a...   22.5029   -0.026960   
198  KLHL7_FL_EPHA3_trimmed_relaxed_rank_002_alphaf...    8.0840   -0.160682   
199  WDTC1_FL_EPHA3_trimmed_relaxed_rank_003_alphaf...   21.7062   -0.107934   

     avg_IF_rep  avg_IF_sol  avg_IF_ele

In [5]:
# Load the Xgboost model
final_model = joblib.load(f"{path_to_code}")
# Use the loaded model and threshold for predictions
model_scores = final_model.predict_proba(X)[:, 1]
# Load the custom threshold from metadata
threshold = 0.5
predicted_label = (model_scores >= threshold).astype(int)

In [6]:
# Create the initial results dataframe
results = pd.DataFrame({
    'File_name': X_df['File_name'],
    'score': model_scores,
    'predicted_label': predicted_label
})

def label_to_prediction(label):
    return 'interacting' if label == 1 else 'not interacting'

results['prediction'] = results['predicted_label'].apply(label_to_prediction)
###Group all 5 models together based on name
results['common_name'] = results['File_name'].apply(lambda x: x.split("_relaxed_")[0])

# Rank by score within each common_name group
results['prob_rank'] = results.groupby('common_name')['score'].rank(method='first', ascending=False).astype(int)

# Filter rows where prob_rank is 1
filtered_df = results[results['prob_rank'] == 1]

final_df = filtered_df.drop(columns=['prob_rank', 'common_name'])
print(final_df)

                                             File_name     score  \
6    KLHDC2_trimmed_EPHA3_trimmed_relaxed_rank_005_...  0.009595   
9    KBTBD4_EPHA3_trimmed_relaxed_rank_001_alphafol...  0.059750   
22   KLHL7_FL_EPHA3_trimmed_relaxed_rank_001_alphaf...  0.409939   
25   FBXO44_FL_EPHA3_trimmed_relaxed_rank_001_alpha...  0.458261   
26   FBXL2_FL_EPHA3_trimmed_relaxed_rank_001_alphaf...  0.059631   
28   DET1_FL_EPHA3_trimmed_relaxed_rank_005_alphafo...  0.003983   
37   DCAF5_FL_EPHA3_trimmed_relaxed_rank_005_alphaf...  0.022107   
42   KLHL6_FL_EPHA3_trimmed_relaxed_rank_001_alphaf...  0.066517   
66   WDTC1_FL_EPHA3_trimmed_relaxed_rank_001_alphaf...  0.107268   
73   TRPC4AP_FL_EPHA3_trimmed_relaxed_rank_001_alph...  0.009680   
77   FBXW7_trimmed_EPHA3_trimmed_relaxed_rank_002_a...  0.063365   
78   CRBN_trimmed_EPHA3_trimmed_relaxed_rank_003_al...  0.004094   
80   DDB2_FL_EPHA3_trimmed_relaxed_rank_001_alphafo...  0.005054   
87   DCAF5_trimmed_EPHA3_trimmed_relaxed_rank_00

In [8]:
####The score is in the column score and the predicted label based on a 0.5 threshold is under the column predicted_label###
filtered_df.to_csv(f"{path_to_AF_models}/E3_EPHA3_DB2022_PPIscreenML_features_results.csv", mode='w', header=True, index=False)

